In [ ]:
#workdir_ = '/glade/work/juliob/'
import sys
#######################################
# Leave this for now. But it should change to better
# method as here:
import os
This_module_path = os.getcwd()  #os.path.dirname(os.path.abspath(__file__))
workdir_ = os.path.join(This_module_path, '../../' )
# sys.path.append(utils_path)
# print( f" a path added in {__name__} {utils_path} ")

print( f" In {__name__} we have This_module_path={This_module_path} " )
print( f" In {__name__} we have workdir_={workdir_} " )
########################################
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
#sys.path.append(workdir_ + 'PyRegridding/Regridder/')
sys.path.append(workdir_ + 'PyRegridding/Utils/')
sys.path.append(workdir_ + 'myPythonTools/Plotting/')

# Own local packages
import AveragingUtils as Av
#import VertRegridFlexLL as Vrg  # This is toxic for some reason
import PlotUtil as Pu
import utils as uti
import validation_data as Val
import var_A_x_B as vAB
import MakePressures as MkP
import GridUtils as GrU
import LatLonMaps as LL

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( LL )
importlib.reload( uti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(vAB)
importlib.reload(MkP)


In [ ]:


X=xr.open_dataset('/glade/campaign/cgd/amp/juliob/amwg_dev/Concordiasi/cciasi_flux_6h.nc')

In [ ]:
X

In [ ]:
nobs = X.number_of_observations.values
print(np.shape(nobs))
plt.plot( np.sum( nobs, axis=(1,2) ) )

In [ ]:
print(X.time.values)

In [ ]:

#plt.contourf( X.longitude, X.latitude, X.absolute_momentum_flux.values[156,:,:] )
plt.contourf( X.longitude, X.latitude,X.number_of_observations.values[156,:,:] ,levels=[10,20,50])
plt.colorbar()


In [ ]:
from datetime import datetime, timedelta

def day_of_year_to_date(year, day_of_year):
    # Create a date object for January 1st of the given year
    jan_first = datetime(year, 1, 1)
    # Add the day_of_year to this date (subtract 1 because day_of_year is 1-based)
    target_date = jan_first + timedelta(days=day_of_year - 1)
    return target_date


In [ ]:
date=day_of_year_to_date(2010, X.time[364].values)
print(date.strftime("%Y-%m-%d:%H"))

In [ ]:

amf=X.absolute_momentum_flux.values
nob=X.number_of_observations.values
lat=X.latitude.values
lon=X.longitude.values

In [ ]:
amf.shape

In [ ]:
plt.plot( nob.flatten(),'.' )
plt.ylim(0,10)

amf0=np.where( nob>=4 , amf, 0. )#np.nan )

In [ ]:
amf0r=np.roll(amf0,shift=72,axis=2)
nobr=np.roll(nob,shift=72,axis=2)

lat_e=0.5*(lat[1:]+lat[:-1])
print(lat_e.shape)

In [ ]:

mflv=np.linspace(0,60,num=21)
#plt.contourf( lon,lat,np.sum(amf0r[120:161,:,:],axis=0), levels=mflv)
plt.contourf( lon,lat,np.sum(nobr[120:161,:,:],axis=0), levels=mflv)
#plt.contour( nob[156,:,:], levels=31, colors='black' )
plt.ylim(-90,90)
plt.colorbar()

Create cciasi scrip file 
lat_e=-88.75+np.arange(19)*2.5
lon_e=1.25+np.arange(143)*2.5

x | x .... x | x 
0             360

In [ ]:
import esmpy as ESMF
# Import packages 
import sys

# import modules in other directories
sys.path.append('../../PyRegridding/Regridder/')
import scripGen as SG
import esmfRegrid as erg

import importlib
importlib.reload( SG )

#Rdry = Con.Rdry() # 

In [ ]:
# Define your longitude values
#lon = np.arange(1.25, 361.25, 2.5)
lon = np.arange( 0., 360., 2.5 )
#lat = -88.75 + np.arange(19)  # Example latitudes for the sake of completeness
lat = -87.5 + np.arange(19)*2.5
dso=SG.latlon_to_scrip( longitudes=lon, latitudes=lat, file_out='CCIASI_scrip.nc',  SkipAreaCheck=True )





In [ ]:
# Create a grid
srcgrid = ESMF.Grid(np.array([len(lat), len(lon)]), staggerloc=ESMF.StaggerLoc.CORNER, coord_sys=ESMF.CoordSys.SPH_DEG)

# Define the coordinates for the grid
srcgrid_corner_lon = srcgrid.get_coords(0, staggerloc=ESMF.StaggerLoc.CORNER)
srcgrid_corner_lat = srcgrid.get_coords(1, staggerloc=ESMF.StaggerLoc.CORNER)

# Assign values to the grid corners
# This is a simplification; usually, you'd have more complex logic to assign these
srcgrid_corner_lon[:, :] = np.roll(lon, -1)  # Ensure cyclic nature
srcgrid_corner_lat[:, :] = lat

# Define source field on the grid
srcfield = ESMF.Field(srcgrid, name='srcfield')

# Define the destination grid similarly...

# Perform regridding
regrid = ESMF.Regrid(srcfield, dstfield, regrid_method=ESMF.RegridMethod.BILINEAR)
regrid(srcfield, dstfield)